# Performance RNN
## Douglas Eck
This notebook shows you how to generate new performed compositions from a trained Performance RNN model. You'll see how to download a bundle containing a pre-trained model, instantiate and initialize the model and generate new polyphonic performances. The notebook also shows some hyperparameters useful for controlling generation, such as ``temperature``.

To hear the performances it is useful to have fluidsynth and pyfluidsynth installed. One way to install fluidsynth on OS X is via homebrew: ``brew install fluidsynth``.  One way ot install pyfluidsynth is via pip: ``pip install pyfluidsynth``.

In [107]:
import os
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

import magenta.music as mm
from magenta.music import midi_io

# Constants.
DEFAULT_SAMPLE_RATE = 44100
SAMPLE_MULT = 1
DURATION = 5
TEMP = 0.6
BUNDLE_DIR = '/Users/wangan/Documents/calhacks2017/magenta/'
MODEL_NAME = 'multiconditioned_performance_with_dynamics'
BUNDLE_NAME = MODEL_NAME + '.mag'

In [88]:
from magenta.music import midi_synth
import IPython

def modified_play_sequence(sequence,
                  synth=midi_synth.synthesize,
                  sample_rate=DEFAULT_SAMPLE_RATE,
                  **synth_args):
  """Creates an interactive player for a synthesized note sequence.
  This function should only be called from a Jupyter notebook.
  Args:
    sequence: A music_pb2.NoteSequence to synthesize and play.
    synth: A synthesis function that takes a sequence and sample rate as input.
    sample_rate: The sample rate at which to synthesize.
    **synth_args: Additional keyword arguments to pass to the synth function.
  """

  array_of_floats = synth(sequence, sample_rate=sample_rate, **synth_args)

  # Cut out first second of audio
  one_second_audio = int(sample_rate * 2)
  array_of_floats = array_of_floats[one_second_audio:]
  IPython.display.display(IPython.display.Audio(array_of_floats, rate=sample_rate))

In [110]:
mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)
bundle = mm.sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))
generator_map = performance_sequence_generator.get_generator_map()
generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)
generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = TEMP  # Higher is more random; 1.0 is default. 
generate_section = generator_options.generate_sections.add(start_time=0, end_time=DURATION)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

# Play and view this masterpiece.
mm.plot_sequence(sequence)
# audio_object = modified_play_sequence(sequence, mm.midi_synth.fluidsynth, sample_rate=DEFAULT_SAMPLE_RATE * SAMPLE_MULT)
# mm.play_sequence(sequence, mm.midi_synth.fluidsynth, sample_rate=DEFAULT_SAMPLE_RATE * SAMPLE_MULT)

# sample_rate=DEFAULT_SAMPLE_RATE * SAMPLE_MULT
array_of_floats = mm.midi_synth.fluidsynth(sequence, sample_rate=sample_rate)
midi_obj = midi_io.sequence_proto_to_pretty_midi(sequence)
midi.fluidsynth(fs=sample_rate, sf2_path=sf2_path).start()

one_second_audio = int(sample_rate * 2)
array_of_floats = array_of_floats[one_second_audio:]

print(midi_obj.midi_file)

# midi_file = open(midi_obj.midi_file, "w")
# midi_file.write(array_of_floats)
# midi_file.flush()
# midi_file.close()

IPython.display.Audio(array_of_floats, rate=sample_rate, autoplay=True)

INFO:tensorflow:Restoring parameters from /var/folders/9b/k4l4q4w15mz_q8h5c5qj1b2m0000gn/T/tmpiem5Sf/model.ckpt
INFO:tensorflow:Need to generate 399 more steps for this sequence, will try asking for 160 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -283.874969 


/Users/wangan/anaconda/envs/magenta/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: ResizeTool is removed in Bokeh 0.12.7, adding it is a no-op. In the future, accessing ResizeTool will be an error
  warn(message)


Loading BokehJS ...

AttributeError: 'PrettyMIDI' object has no attribute 'midi_file'